In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# n has to be a power of two
k = 8
n = 2**k

## Define a vector to interplate
#v = np.zeros(n)
#v[n//4:3*n//4] = 1
z = np.arange(n) / n
v = 2 * np.sin(2 * np.pi * z) + 0.5 * np.sin(2 * np.pi * (8 * z + 0.5))

In [ ]:
# Discrete fourier transform as defined in the lecture on slide 36
def dft(v):
    flop_count = 0
    n = len(v)
    omega_bar = np.exp(-2*np.pi * 1j / n)
    c = np.zeros(n, np.complex128)
    for k in range(n):
        for j in range(n):
            c[k] += v[j] * omega_bar ** (j*k)
            flop_count += 4
        c[k] /= n
    return c, flop_count

# Inverse fourier transform as defined in assignment 3
def ifft(c):
    n = len(c)
    if n == 1:
        return c, 0
    m = n//2
    c_even = c[0::2]
    c_odd = c[1::2]
    z_1, f_1 = ifft(c_even)
    z_2, f_2 = ifft(c_odd)
    flop_count = f_1 + f_2
    omega = np.exp(2*np.pi*1j/n)
    v = np.zeros(n, np.complex128)
    for j in range(m):
        v[j] = z_1[j] + omega**j * z_2[j]
        v[m+j] = z_1[j] - omega**j * z_2[j]
        flop_count += 6
    return v, flop_count

# Use the result from assignment 2/c
def fft(v):
    n = len(v)
    ifft_result, flop_counter = ifft(np.conjugate(v))
    return 1 / n * np.conjugate(ifft_result), flop_counter

# Compute fft and dft and compare result and flop counter
c,f  = fft(v)
c_, f_ = dft(v)
print(f, f_)
print(np.linalg.norm(c - c_))

In [ ]:
# Extract amplitudes and phases from the fourier transform
# In a perfect sine wave `A * sin(2 * pi * (f * t + phi))`, 
# A is the amplitude, f is the frequency and phi is the phase.
def plot_spectrum(c):
    n = len(c)
    k = np.zeros(n, np.int64)
    for k_ in range(-n//2+1, n//2+1):
        k[k_] = k_
    freq = np.arange(n//2)
    amplitudes = np.zeros(freq.shape)
    phases = np.zeros(freq.shape, np.complex128)
    amplitudes[0] = c[0].real
    for m in range(1,n//2):
        amplitudes[m] = np.abs(c[k[-m]]) + np.abs(c[k[m]])
        if amplitudes[m] > 1e-10:
            phases[m] = np.angle(c[k[-m]]) - np.angle(c[k[m]])
            phases[m] /= (2*np.pi)
            phases[m] -= 0.5
            phases[m] /= -2
    fig, axes = plt.subplots(1, 2)
    axes[0].set_title("Amplitudes")
    axes[0].scatter(freq, amplitudes.real)
    axes[1].set_title("Phases")
    axes[1].scatter(freq, phases.real)
    plt.tight_layout()
    plt.show()

# Interpret Fourier transformed data as frequencies up to n/2
def plot_fourier(v, c):
    n = len(v)
    assert n == len(c)
    t = np.linspace(0, 1, 101)
    p = c[0] * np.ones(101, np.complex128)
    if n < 4:
        plt.plot(t, p, label="0")
    for k in range(1, n//2+1):
        f = c[k] * np.exp(2*np.pi*1j*k*t) + c[-k] * np.exp(-2*np.pi*1j*k*t)
        if n < 4:
            plt.plot(t, f, label=f"{k}")
        p += f
        
    plt.plot(t, p.real, label="Re(p)")
    plt.plot(t, p.imag, label="Im(p)")
    
    z = np.arange(n) / n
    if n < 17:
        plt.scatter(z, v)
    plt.legend()
    plt.show()
    
plot_spectrum(c)
plot_fourier(v, c)